# Building graphs with Metric Learning (and an introduction to the new pytorch lightning-based framework)

In [21]:
from functools import partial

import torch

from gnn_tracking.training.ml import MLModule
from gnn_tracking.models.graph_construction import GraphConstructionFCNN
from gnn_tracking.metrics.losses import GraphConstructionHingeEmbeddingLoss
from pytorch_lightning import Trainer
from gnn_tracking.utils.loading import TrackingDataModule

## Step 1: Configuring the data

The configuration for train/val/test data and its dataloader is held in a `LightningDataModule`.

In [22]:
dm = TrackingDataModule(
    train=dict(
        dirs=["/Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/"],
        stop=5,
    ),
    val=dict(
        dirs=["/Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/"],
        start=5,
        stop=10,
    ),
    # could also configure a 'test' set here
)

Other keys allow to configure the loaders (batch size, number of workers, etc.). See the docstring of `TrackingDataModule` for details.

### Details (for understanding)

Note that all of the following will be done implicitly by the `Trainer` and you won't have to worry about it. But if you want to inspect the data, you can do so.

When calling the `setup` method, the `LightningDataModule` initializes instances of `TrackingDataset` (`torch_geometric.Dataset`) for each of these. We can get the corresponding dataloaders by calling `dm.train_dataloader()` and analog for validation and test.

Example:

In [23]:
# This is called by the Trainer automatically and sets up the datasets
dm.setup(stage="fit")  # 'fit' combines 'train' and 'val'
# Now the datasets are available:
dm.datasets

[19:29:55] INFO: DataLoader will load 5 graphs (out of 90 available).
[19:29:55] DEBUG: First graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21025_s0.pt, last graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21053_s0.pt
[19:29:55] INFO: DataLoader will load 5 graphs (out of 90 available).
[19:29:55] DEBUG: First graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21058_s0.pt, last graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21094_s0.pt


{'train': TrackingDataset(5), 'val': TrackingDataset(5)}

For example, we can inspect the first element of the training dataset:

In [6]:
data = dm.datasets["train"][0]

To get the corresponding dataloaders, use one of the methods (but again, you probalby won't need to):

In [7]:
dm.train_dataloader(), dm.val_dataloader()

(<torch_geometric.loader.dataloader.DataLoader at 0x147d2f090>,
 <torch_geometric.loader.dataloader.DataLoader at 0x15815d950>)

## Step 2: Configuring a model

We write a normal `torch.nn.Module`.

In [8]:
model = GraphConstructionFCNN(in_dim=14, out_dim=8, depth=5, hidden_dim=64)

### Details (for understanding)

The only difference is a call `self.save_hyperparameters()` in `__init__` (and the inheritance from `HyperparametersMixin`). This allows us to save all hyperparameters:

In [24]:
model.hparams

"beta":       0.4
"depth":      5
"hidden_dim": 64
"in_dim":     14
"out_dim":    8

Note how `beta=0.4` was saved despite not being specified explicitly (it's set as a default parameter).

As always, you can simply evaluate the `model` on a piece of data:

In [25]:
out = model(data)

## Step 3: Configuring loss functions and metrics

The pytorch model is bundled together with a set of loss functions (just one here), that we backpropagate from in the training step, and a set of metrics. Together, these components make up the `LightningModule` that we pass to the pytorch lightning `Trainer` for training.

If you were familiar with our previous `TCNTrainer` training class, this `MLModule` now fulfills (almost) the exact same role.

In [11]:
lmodel = MLModule(
    model=model,
    loss_fct=GraphConstructionHingeEmbeddingLoss(max_num_neighbors=10),
    lw_repulsive=0.5,  # loss weight, see below
    optimizer=partial(torch.optim.Adam, lr=1e-4),
)

[19:25:53] DEBUG: Got obj of type <class 'gnn_tracking.models.graph_construction.GraphConstructionFCNN'>, assuming I have to save hyperparameters
[19:25:53] DEBUG: Saving hyperperameters {'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN', 'init_args': {'in_dim': 14, 'hidden_dim': 64, 'out_dim': 8, 'depth': 5, 'beta': 0.4}}
[19:25:53] DEBUG: Got obj of type <class 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss'>, assuming I have to save hyperparameters
[19:25:53] DEBUG: Saving hyperperameters {'class_path': 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss', 'init_args': {'r_emb': 0.002, 'max_num_neighbors': 10, 'attr_pt_thld': 0.9, 'p_attr': 1, 'p_rep': 1}}


### Details (for understanding)

Again, all hyperparameters are accessible (even the ones that weren't explicitly specified but only set by default):

In [12]:
lmodel.hparams

"loss_fct":     {'class_path': 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss', 'init_args': {'r_emb': 0.002, 'max_num_neighbors': 10, 'attr_pt_thld': 0.9, 'p_attr': 1, 'p_rep': 1}}
"lw_repulsive": 0.5
"model":        {'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN', 'init_args': {'in_dim': 14, 'hidden_dim': 64, 'out_dim': 8, 'depth': 5, 'beta': 0.4}}

The loss function takes output from the model and the data and returns two separate losses:

In [13]:
loss_fct = GraphConstructionHingeEmbeddingLoss()
loss_fct(
    x=out["H"],
    particle_id=data.particle_id,
    batch=data.batch,
    edge_index=data.edge_index,
    pt=data.pt,
)

{'attractive': tensor(0.0257, grad_fn=<DivBackward0>),
 'repulsive': tensor(0.0028, grad_fn=<SumBackward0>)}

Both parts of the loss functions are combined with the loss weight we have configured above (weight of 1 for attractive, weight of 0.5 for repulsive). All of this is done in `MLModule.get_losses` (returning the total loss and a dictionary of the individual losses):

In [14]:
lmodel.get_losses(out, data)

(tensor(0.0318, grad_fn=<AddBackward0>),
 {'attractive': 0.03146896883845329,
  'repulsive': 0.0006920472951605916,
  'attractive_weighted': 0.03146896883845329,
  'repulsive_weighted': 0.0003460236475802958})

## Step 4: Training

In [33]:
trainer = Trainer(max_epochs=1, accelerator="cpu", log_every_n_steps=1)
trainer.fit(model=lmodel, train_dataloaders=dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
Missing logger folder: /Users/fuchur/Documents/22/git_sync/gnn_tracking/lightning_logs
[19:37:33] INFO: DataLoader will load 5 graphs (out of 90 available).
[19:37:33] DEBUG: First graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21025_s0.pt, last graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21053_s0.pt
[19:37:33] INFO: DataLoader will load 5 graphs (out of 90 available).
[19:37:33] DEBUG: First graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21058_s0.pt, last graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21094_s0.pt

  | Nam

Sanity Checking: 0it [00:00, ?it/s]

/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 6789. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8459. To avoid any miscalculations, use `self.log(..., batch_size=batch_siz

           Validation epoch=0            
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┓
┃ Metric              ┃   Value ┃ Error ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━┩
│ attractive          │ 0.03392 │   nan │
│ attractive_weighted │ 0.03392 │   nan │
│ repulsive           │ 0.00078 │   nan │
│ repulsive_weighted  │ 0.00039 │   nan │
│ total               │ 0.03431 │   nan │
└─────────────────────┴─────────┴───────┘

/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 7299. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5946. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8207. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


           Validation epoch=0            
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┓
┃ Metric              ┃   Value ┃ Error ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━┩
│ attractive          │ 0.03069 │   nan │
│ attractive_weighted │ 0.03069 │   nan │
│ repulsive           │ 0.00077 │   nan │
│ repulsive_weighted  │ 0.00038 │   nan │
│ total               │ 0.03107 │   nan │
└─────────────────────┴─────────┴───────┘

`Trainer.fit` stopped: `max_epochs=1` reached.


## Restoring a pre-trained model

Take a look at the `lightning_logs` directory:

In [36]:
! ls lightning_logs

version_0
version_0


In [40]:
! ls lightning_logs/version_0/checkpoints

epoch=0-step=5.ckpt


Navigate to one of the versions and take a look at the `hparams.yaml` file. It should contain exactly the hyperparameters from the run.


In [38]:
! cat lightning_logs/version_0/hparams.yaml

model:
  class_path: gnn_tracking.models.graph_construction.GraphConstructionFCNN
  init_args:
    in_dim: 14
    hidden_dim: 64
    out_dim: 8
    depth: 5
    beta: 0.4
lw_repulsive: 0.5
loss_fct:
  class_path: gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss
  init_args:
    r_emb: 0.002
    max_num_neighbors: 10
    attr_pt_thld: 0.9
    p_attr: 1
    p_rep: 1


We can bring back the trained model by loading one of the checkpoints:

In [42]:
restored_model = MLModule.load_from_checkpoint(
    "lightning_logs/version_0/checkpoints/epoch=0-step=5.ckpt"
)

[19:39:17] DEBUG: Got {'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN', 'init_args': {'in_dim': 14, 'hidden_dim': 64, 'out_dim': 8, 'depth': 5, 'beta': 0.4}}, assuming I have to load
[19:39:17] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[19:39:17] DEBUG: Got {'class_path': 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss', 'init_args': {'r_emb': 0.002, 'max_num_neighbors': 10, 'attr_pt_thld': 0.9, 'p_attr': 1, 'p_rep': 1}}, assuming I have to load
[19:39:17] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses


Note how we didn't have to specify any the hyperparameters again.

However, we can easily change some of them by adding them as additional keyword arguments.

In [45]:
restored_model_modified = MLModule.load_from_checkpoint(
    "lightning_logs/version_0/checkpoints/epoch=0-step=5.ckpt",
    lw_repulsive=0.1,
    loss_fct=GraphConstructionHingeEmbeddingLoss(max_num_neighbors=5),
)

[19:40:11] DEBUG: Got {'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN', 'init_args': {'in_dim': 14, 'hidden_dim': 64, 'out_dim': 8, 'depth': 5, 'beta': 0.4}}, assuming I have to load
[19:40:11] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[19:40:11] DEBUG: Got obj of type <class 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss'>, assuming I have to save hyperparameters
[19:40:11] DEBUG: Saving hyperperameters {'class_path': 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss', 'init_args': {'r_emb': 0.002, 'max_num_neighbors': 5, 'attr_pt_thld': 0.9, 'p_attr': 1, 'p_rep': 1}}


Note that you cannot modify the model architecture however (but you could in principle change the `beta` parameter of the residual connections).

## Running all of this from the command line

All of the following can be achieved by running the following command:

```bash
python3 gnn_tracking/trainers/run.py fit --model configs/model.yml --data configs/data.yml  --trainer.accelerator cpu --trainer.accelerator cpu
```

with the data config file

```yaml
train:
  dirs:
    - /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/
  stop: 5
test:
  dirs:
    - /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/
  star: 10
  stop: 15
val:
  dirs:
    - /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/
  start: 5
  stop: 10
```

and model config file:

```
class_path: gnn_tracking.training.ml.MLModule
init_args:
  model:
    class_path: gnn_tracking.models.graph_construction.GraphConstructionFCNN
    init_args:
      in_dim: 14
      out_dim: 8
      hidden_dim: 512
      depth: 5
  lw_repulsive: 0.5
  loss_fct:
    class_path: gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss
    init_args: {}
  optimizer:
    class_path: torch.optim.Adam
    init_args:
      lr: 0.0001
```

To quickly override one of the options, you can simply add them to the command line, e.g., `--model.init_args.lw_repulsive=0.1` or `--model.model.init_args.depth=6`.